In [1]:
import json
from authorization import SpotifyAuth

In [2]:
with open('client-secrets-rnr.json','r+') as secrets_file:
    secrets = json.load(secrets_file)

client_id = secrets["client_id"]
client_secret = secrets["client_secret"]
redirect_url = secrets["redirect_url"]

In [5]:
spotify_authorisation = SpotifyAuth(client_id=client_id, client_secret=client_secret, redirect_url=redirect_url)
spotify_authorisation.set_access_token_from_file('access_token.json')

# Get User Details

In [7]:
import spotify_functions as spotify

In [22]:
spotify.get_user_details(spotify_authorisation.get_access_token())

{'display_name': 'sdspot2034',
 'external_urls': {'spotify': 'https://open.spotify.com/user/sdspot2034'},
 'href': 'https://api.spotify.com/v1/users/sdspot2034',
 'id': 'sdspot2034',
 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b8280153af8b79b3144b24f1291',
   'height': 64,
   'width': 64},
  {'url': 'https://i.scdn.co/image/ab6775700000ee8580153af8b79b3144b24f1291',
   'height': 300,
   'width': 300}],
 'type': 'user',
 'uri': 'spotify:user:sdspot2034',
 'followers': {'href': None, 'total': 9},
 'country': 'IN',
 'product': 'premium',
 'explicit_content': {'filter_enabled': False, 'filter_locked': False},
 'email': 'shreyandas2034@gmail.com'}

In [9]:
import datetime
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
# print(yesterday.timestamp()*1e3)
cdc_time = int(yesterday.timestamp()*1e3)

# Get All Songs Played in the last one day

In [11]:
import requests

In [12]:
token = spotify_authorisation.get_access_token()

result = requests.get("https://api.spotify.com/v1/me/player/recently-played", 
        headers={"Authorization": "Bearer " + token}, params={"after": str(cdc_time), "limit":"50"})

In [14]:
import pprint
pprint.pprint(json_results)

{'cursors': {'after': '1722440198028', 'before': '1722401651300'},
 'href': 'https://api.spotify.com/v1/me/player/recently-played?after=1722358375577&limit=50',
 'items': [{'context': None,
            'played_at': '2024-07-31T15:36:38.028Z',
            'track': {'album': {'album_type': 'album',
                                'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5pKCCKE2ajJHZ9KAiaK11H'},
                                             'href': 'https://api.spotify.com/v1/artists/5pKCCKE2ajJHZ9KAiaK11H',
                                             'id': '5pKCCKE2ajJHZ9KAiaK11H',
                                             'name': 'Rihanna',
                                             'type': 'artist',
                                             'uri': 'spotify:artist:5pKCCKE2ajJHZ9KAiaK11H'}],
                                'available_markets': ['AR',
                                                      'AU',
                                    

## EDA over recent history

### Number of songs played

In [15]:
len(json_results['items'])

50

In [16]:
type(json_results['items'])

list

### Fetch details of a song

#### See available data for a song

In [17]:
json_results['items'][0]

{'track': {'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5pKCCKE2ajJHZ9KAiaK11H'},
     'href': 'https://api.spotify.com/v1/artists/5pKCCKE2ajJHZ9KAiaK11H',
     'id': '5pKCCKE2ajJHZ9KAiaK11H',
     'name': 'Rihanna',
     'type': 'artist',
     'uri': 'spotify:artist:5pKCCKE2ajJHZ9KAiaK11H'}],
   'available_markets': ['AR',
    'AU',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    'HN',
    'HK',
    'HU',
    'IS',
    'IE',
    'IT',
    'LV',
    'LT',
    'LU',
    'MY',
    'MT',
    'MX',
    'NL',
    'NZ',
    'NI',
    'NO',
    'PA',
    'PY',
    'PE',
    'PH',
    'PL',
    'PT',
    'SG',
    'SK',
    'ES',
    'SE',
    'CH',
    'TW',
    'TR',
    'UY',
    'GB',
    'AD',
    'LI',
    'MC',
    'ID',
    'JP',
    'TH',
    'VN',
    'RO',
    'IL'

In [18]:
json_results['items'][0].keys()

dict_keys(['track', 'played_at', 'context'])

In [19]:
json_results['items'][0]['track']

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5pKCCKE2ajJHZ9KAiaK11H'},
    'href': 'https://api.spotify.com/v1/artists/5pKCCKE2ajJHZ9KAiaK11H',
    'id': '5pKCCKE2ajJHZ9KAiaK11H',
    'name': 'Rihanna',
    'type': 'artist',
    'uri': 'spotify:artist:5pKCCKE2ajJHZ9KAiaK11H'}],
  'available_markets': ['AR',
   'AU',
   'AT',
   'BE',
   'BO',
   'BR',
   'BG',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DK',
   'DO',
   'DE',
   'EC',
   'EE',
   'SV',
   'FI',
   'FR',
   'GR',
   'GT',
   'HN',
   'HK',
   'HU',
   'IS',
   'IE',
   'IT',
   'LV',
   'LT',
   'LU',
   'MY',
   'MT',
   'MX',
   'NL',
   'NZ',
   'NI',
   'NO',
   'PA',
   'PY',
   'PE',
   'PH',
   'PL',
   'PT',
   'SG',
   'SK',
   'ES',
   'SE',
   'CH',
   'TW',
   'TR',
   'UY',
   'GB',
   'AD',
   'LI',
   'MC',
   'ID',
   'JP',
   'TH',
   'VN',
   'RO',
   'IL',
   'ZA',
   'SA',
   'AE',
   'BH',
   'QA',
   'OM',
   'KW',
   'EG',
   '

In [20]:
list(json_results['items'][0]['track'].keys())

['album',
 'artists',
 'available_markets',
 'disc_number',
 'duration_ms',
 'explicit',
 'external_ids',
 'external_urls',
 'href',
 'id',
 'is_local',
 'name',
 'popularity',
 'preview_url',
 'track_number',
 'type',
 'uri']

In [21]:
track_details = {k: v for k, v in json_results['items'][0]['track'].items() if k not in ('available_markets')}

if 'album' in track_details:
    del track_details['album']['images']
    del track_details['album']['available_markets']

track_details


{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5pKCCKE2ajJHZ9KAiaK11H'},
    'href': 'https://api.spotify.com/v1/artists/5pKCCKE2ajJHZ9KAiaK11H',
    'id': '5pKCCKE2ajJHZ9KAiaK11H',
    'name': 'Rihanna',
    'type': 'artist',
    'uri': 'spotify:artist:5pKCCKE2ajJHZ9KAiaK11H'}],
  'external_urls': {'spotify': 'https://open.spotify.com/album/5pLlGJrxuQO3jMoQe1XxZY'},
  'href': 'https://api.spotify.com/v1/albums/5pLlGJrxuQO3jMoQe1XxZY',
  'id': '5pLlGJrxuQO3jMoQe1XxZY',
  'name': 'Unapologetic (Deluxe)',
  'release_date': '2012-12-11',
  'release_date_precision': 'day',
  'total_tracks': 17,
  'type': 'album',
  'uri': 'spotify:album:5pLlGJrxuQO3jMoQe1XxZY'},
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5pKCCKE2ajJHZ9KAiaK11H'},
   'href': 'https://api.spotify.com/v1/artists/5pKCCKE2ajJHZ9KAiaK11H',
   'id': '5pKCCKE2ajJHZ9KAiaK11H',
   'name': 'Rihanna',
   'type': 'artist',
   'uri': 'spotif